In [ ]:


from typing import List, Tuple

import astropy.coordinates as ac
import astropy.time as at

from dsa2000_assets.array_constraints.array_constraint_content import ArrayConstraintsV6
from dsa2000_fm.array_layout.geo_constraints import RegionSampler
from dsa2000_fm.array_layout.psf_evaluation import Results


def plot_solution(plot_folder: str, antennas: ac.EarthLocation, obstime: at.Time, array_location: ac.EarthLocation,
                  aoi_data: List[Tuple[RegionSampler, float]],
                  constraint_data: List[Tuple[RegionSampler, float]]):
    # Plot along with regions
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    # array_constraint = ArrayConstraintV2()
    # aoi_data = array_constraint.get_area_of_interest_regions()
    # constraint_data = array_constraint.get_constraint_regions()
    for sampler, buffer in aoi_data:
        # sampler.info()
        sampler.plot_region(ax=ax, color='blue')
    for sampler, buffer in constraint_data:
        sampler.plot_region(ax=ax, color='none')

    # ax.scatter(antennas_enu[:, 0], antennas_enu[:, 1], s=1, c='green', alpha=0.5, marker='.')
    ax.scatter(antennas.geodetic.lon.deg, antennas.geodetic.lat.deg, s=1, c='green', alpha=0.5, marker='.')
    ax.set_xlabel('Longitude [deg]')
    ax.set_ylabel('Latitude [deg]')
    ax.set_title('Antenna layout')
    ax.set_xlim(-114.6, -114.3)
    ax.set_ylim(39.45, 39.70)
    # fig.savefig(os.path.join(plot_folder, f'antenna_solution.png'))
    plt.show()

    # Plot violations
    for idx, point in enumerate(antennas):
        for sampler, buffer in constraint_data:
            (px, py), dist = sampler.closest_approach(point.geodetic.lon.deg, point.geodetic.lat.deg)
            earth_radius = np.linalg.norm(point.get_itrs().cartesian.xyz.to(au.m).value)
            dist = np.pi / 180 * dist * earth_radius

            if dist < buffer:
                print('Agree')
                sampler.info()
                fig, ax = plt.subplots(1, 1, figsize=(6, 6))
                sampler.plot_region(ax=ax, color='none')
                ax.scatter(px, py, c='g')
                ax.scatter(point.geodetic.lon.deg, point.geodetic.lat.deg, c='b')
                bbox = min(point.geodetic.lon.deg, px), max(point.geodetic.lon.deg, px), min(point.geodetic.lat.deg,
                                                                                             py), max(
                    point.geodetic.lat.deg, py)
                ax.set_xlim(bbox[0] - 0.005, bbox[1] + 0.005)
                ax.set_ylim(bbox[2] - 0.005, bbox[3] + 0.005)
                ax.set_title(f"{dist} {buffer}")
                plt.show()


results_file = "./results.json"
results = Results.parse_file(results_file)

array_constraint = ArrayConstraintsV6()
aoi_data = array_constraint.get_area_of_interest_regions()
# merge AOI's
merged_aoi_sampler = RegionSampler.merge([s for s, _ in aoi_data])
merged_buffer = max([b for _, b in aoi_data])
aoi_data = [(merged_aoi_sampler, merged_buffer)]
constraint_data = array_constraint.get_constraint_regions()

evaluation = min(results.evaluations, key=lambda x: x.cost)

plot_solution('./', evaluation.antennas, results.obstime, results.array_location, aoi_data, constraint_data)

with open('solution.txt', 'w') as f:
    f.write('#X,Y,Z\n')
    for antenna in evaluation.antennas:
        f.write(f"{antenna.x.to('m').value},{antenna.y.to('m').value},{antenna.z.to('m').value}\n")


In [ ]:

from typing import List, Tuple

import astropy.coordinates as ac
import astropy.time as at
import astropy.units as au
import numpy as np
import pylab as plt

from dsa2000_assets.array_constraints.array_constraint_content import ArrayConstraintsV5
from dsa2000_fm.array_layout.geo_constraints import RegionSampler
from dsa2000_fm.array_layout.psf_evaluation import Results


def plot_solution(plot_folder: str, antennas: ac.EarthLocation, obstime: at.Time, array_location: ac.EarthLocation,
                  aoi_data: List[Tuple[RegionSampler, float]],
                  constraint_data: List[Tuple[RegionSampler, float]]):
    # Plot along with regions
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    # array_constraint = ArrayConstraintV2()
    # aoi_data = array_constraint.get_area_of_interest_regions()
    # constraint_data = array_constraint.get_constraint_regions()
    for sampler, buffer in aoi_data:
        # sampler.info()
        sampler.plot_region(ax=ax, color='blue')
    for sampler, buffer in constraint_data:
        sampler.plot_region(ax=ax, color='none')

    # ax.scatter(antennas_enu[:, 0], antennas_enu[:, 1], s=1, c='green', alpha=0.5, marker='.')
    ax.scatter(antennas.geodetic.lon.deg, antennas.geodetic.lat.deg, s=1, c='green', alpha=0.5, marker='.')
    ax.set_xlabel('Longitude [deg]')
    ax.set_ylabel('Latitude [deg]')
    ax.set_title('Antenna layout')
    ax.set_xlim(-114.6, -114.3)
    ax.set_ylim(39.45, 39.70)
    # fig.savefig(os.path.join(plot_folder, f'antenna_solution.png'))
    plt.show()

    # Plot violations
    for idx, point in enumerate(antennas):
        for sampler, buffer in constraint_data:
            (px, py), dist = sampler.closest_approach(point.geodetic.lon.deg, point.geodetic.lat.deg)
            earth_radius = np.linalg.norm(point.get_itrs().cartesian.xyz.to(au.m).value)
            dist = np.pi / 180 * dist * earth_radius

            if dist < buffer:
                print('Agree')
                sampler.info()
                fig, ax = plt.subplots(1, 1, figsize=(6, 6))
                sampler.plot_region(ax=ax, color='none')
                ax.scatter(px, py, c='g')
                ax.scatter(point.geodetic.lon.deg, point.geodetic.lat.deg, c='b')
                bbox = min(point.geodetic.lon.deg, px), max(point.geodetic.lon.deg, px), min(point.geodetic.lat.deg,
                                                                                             py), max(
                    point.geodetic.lat.deg, py)
                ax.set_xlim(bbox[0] - 0.005, bbox[1] + 0.005)
                ax.set_ylim(bbox[2] - 0.005, bbox[3] + 0.005)
                ax.set_title(f"{dist} {buffer}")
                plt.show()


results_file = "./results.json"
results = Results.parse_file(results_file)

array_constraint = ArrayConstraintsV5()
aoi_data = array_constraint.get_area_of_interest_regions()
# merge AOI's
merged_aoi_sampler = RegionSampler.merge([s for s, _ in aoi_data])
merged_buffer = max([b for _, b in aoi_data])
aoi_data = [(merged_aoi_sampler, merged_buffer)]
constraint_data = array_constraint.get_constraint_regions()

evaluation = min(results.evaluations, key=lambda x: x.cost)

plot_solution('./', evaluation.antennas, results.obstime, results.array_location, aoi_data, constraint_data)

with open('solution.txt', 'w') as f:
    f.write('#X,Y,Z\n')
    for antenna in evaluation.antennas:
        f.write(f"{antenna.x.to('m').value},{antenna.y.to('m').value},{antenna.z.to('m').value}\n")